In [4]:
groups_by_keyword = {}
groups_by_master = {}

start = '<p>'
margin_1 = '<p style="margin-left: 0.79cm">'
margin_2 = '<p style="margin-left: 1.59cm">' # maybe deal with this later...

f = open("keywords.txt", "r")
for line in f:
    if line != '\n':
        line = line.split('</p>')
        if start in line[0]:
            curr_master = line[0].split(start)[1]
            groups_by_keyword[curr_master] = curr_master
            groups_by_master[curr_master] = [curr_master]
        else:
            split_by = margin_1 if margin_1 in line[0] else margin_2
            keyword = line[0].split(split_by)[1]
            groups_by_keyword[keyword] = curr_master
            groups_by_master[curr_master].append(keyword)

In [5]:
import csv

source_tags = []
tag_blacklist = ['Journal', 'Year', 'Number of Citations']
valid_keywords = list(groups_by_keyword.keys())
num_of_citations = ['0', '1-10', '11-20', '21-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-90', '91-100', '101-110', '111-120', '121-130', '131-140', '141-150', '151-160', '161-170', '171-180', '181-190', '191-200', '201-210', '211-220', '221-230', '231-240', '241-250', '251-260' , '261-270', '271-280', '281-290', '291-300', '301-310', '311-320', '321-330', '331-340', '341-350', '351-360', '361-370', '371-380', '381-390', '391-400', '401-410', '411-420', '421-430', '431-440', '441-450', '481-490', '511-520', '561-570', '571-580', '581-590', '591-600', '601-610', '611-620', '631-640', '661-670', '681-690', '701-710', '811-820', '961-970', '1010-1020', '1071-1080', '1181-1190', '1301-1310', '1631-1640', '1831-1840', '4191-4200']
journals = ['Annual Review of Clinical Psychology', 'Attention, Perception, & Psychophysics', 'Behavioral and Brain Scienes', 'Cognitive Psychology', 'Educational Psychologist', 'Nature Neuroscience', 'Neuroscience and Biobehavioral Reviews', 'New Ideas in Psychology', 'Trends in Cognitive Science']

with open('sources.csv') as csv_file:
    csv_file.readline()
    line = csv_file.readline()
    csv_reader = csv.reader(csv_file, delimiter=',')
    
    first_source = next(csv_reader)
    nodes = first_source[1].split('\\')
    tag_set = []
    for keyword in nodes[2:]:
        if keyword in valid_keywords:
            tag_set.append(keyword)
        #tag_set.append(keyword.lower())
        
    for row in csv_reader:
        if row[0] == first_source[0]:
            nodes = row[1].split('\\')
            for keyword in nodes[2:]:
                if keyword not in tag_set and keyword not in tag_blacklist and keyword in valid_keywords:
                    tag_set.append(keyword)
                    #tag_set.append(keyword.lower())
        else:
            source_tags.append(tag_set)
            first_source = row
            tag_set = []
            nodes = row[1].split('\\')
            for keyword in nodes[2:]:
                if keyword not in tag_set and keyword not in tag_blacklist and keyword in valid_keywords:
                    tag_set.append(keyword)
                    #tag_set.append(keyword.lower())

In [78]:
year_collections = {'2007': {}, '2008': {}, '2009': {}, '2010': {}, '2011': {}, '2012': {}, '2013': {}, '2014': {}, '2015': {}, '2016': {}, '2017': {}, '2018': {}}

In [79]:
index = 0
for source in source_tags:
    if source[-1] not in year_collections.keys():
        print(source, index)
    index += 1

no_year_indices = [311, 335, 656]
youth_indices = [27, 532]

['child development', 'children', 'ecological psychology', 'ecological principles', 'education', 'Annual Review of Clinical Psychology', 'mental health', '1-10', 'schools', '2017', 'youth'] 27
['brain networks', 'computational modeling', 'computational models', 'connectionism', 'connectome development', 'longitudinal studies', 'Trends in Cognitive Science', '1-10'] 311
['brain structure', 'frontal cortex', 'medial prefrontal cortex', 'limbic structures', 'posterior cerebral cortex', 'Event perception', 'event simulators', '131-140', 'Social modeling', 'social event knowledge', 'social-affective neuroscience', 'social neuroscience'] 335
['gender', 'gender identity', 'Annual Review of Clinical Psychology', '71-80', 'sexual orientation', '2016', 'youth'] 532
['brain structure', 'limbic structures', 'hippocampus', 'dopamine', 'Annual Review of Clinical Psychology', '421-430', 'psychopathology', 'schizophrenia', 'stress'] 656


In [48]:
import math

### EXTRA CODE ###

#if num_citations in '0':
    #citations = 0
#else:
    #citations = int(num_citations.split('-')[1])
    #citations = math.ceil(citations/100) * 100

In [80]:
index = 0

for source in source_tags:
    num_citations = [keyword for keyword in source if keyword in num_of_citations][0]
    journal = [keyword for keyword in source if keyword in journals]
    if index not in no_year_indices:
        if index in youth_indices:
            source_mod = (source[:-2] + ['youth'], source[-2])
        else:
            source_mod = (source[:-1], source[-1])
            
        for keyword in source_mod[0]:
            if keyword not in num_citations and keyword not in journal:
                year_dict = year_collections[source_mod[1]]
                if num_citations in '0':
                    citations = 0
                else:
                    citations = int(num_citations.split('-')[1])
                    citations = math.ceil(citations/100) * 100
                
                j_entry = journal[0] if journal else 'Not Specified'
                if keyword not in year_dict:
                    year_dict[keyword] = {citations: ([num_citations], [j_entry])}
                else:
                    if citations not in year_dict[keyword]:
                        year_dict[keyword][citations] = ([num_citations], [j_entry])
                    else:
                        year_dict[keyword][citations][0].append(num_citations)
                        year_dict[keyword][citations][1].append(j_entry)
    index += 1

In [81]:
for year, data in year_collections.items():
    traces = []
    for keyword in data.keys():
        if groups_by_keyword[keyword] not in traces:
            traces.append(groups_by_keyword[keyword])
    year_collections[year]['traces'] = traces

In [82]:
year_collections['2016']

{'cognitive systems': {100: (['21-30', '21-30', '11-20', '11-20', '1-10'],
   ['Trends in Cognitive Science',
    'Educational Psychologist',
    'New Ideas in Psychology',
    'Cognitive Psychology',
    'New Ideas in Psychology'])},
 'cognitive control': {100: (['21-30'], ['Trends in Cognitive Science'])},
 'Action-based framework': {100: (['21-30', '1-10'],
   ['Behavioral and Brain Scienes', 'Not Specified'])},
 'action-oriented': {100: (['21-30'], ['Behavioral and Brain Scienes'])},
 'Dynamic systems theory': {100: (['21-30'],
   ['Behavioral and Brain Scienes'])},
 'dynamic systems': {100: (['21-30'], ['Behavioral and Brain Scienes'])},
 'embodied cognition': {100: (['21-30', '1-10', '1-10', '1-10'],
   ['Behavioral and Brain Scienes',
    'New Ideas in Psychology',
    'New Ideas in Psychology',
    'New Ideas in Psychology'])},
 'Evolution': {100: (['21-30', '51-60', '11-20'],
   ['Behavioral and Brain Scienes',
    'Behavioral and Brain Scienes',
    'New Ideas in Psychology']

In [83]:
import plotly
plotly.tools.set_credentials_file(username='squinto', api_key='gdQE35ftpJCY4ZE2G6Fu')

In [84]:
import plotly.plotly as py
import plotly.graph_objs as go

In [85]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from IPython.display import display, HTML

init_notebook_mode(connected=True)

In [214]:
### GRID VERSION ###

from plotly.grid_objs import Grid, Column

# list of years
years = list(year_collections.keys())

columns = []
temp_cols = {}
for year in years:
    data = year_collections[year]
    traces = data['traces']
    for master in traces:
        temp_cols['{year}_{master}'.format(year=year, master=master)] = {'keyword': [], 'journal': [], 'y-val': [], 'citations': []}
        
    for keyword, num_citations in data.items():
        if keyword != 'traces':
            for num, count_info in num_citations.items():
                for counter, num_cites in enumerate(count_info[0]):
                    master = groups_by_keyword[keyword]
                    temp_cols['{year}_{master}'.format(year=year, master=master)]['keyword'].append(keyword+'-'+num_cites)
                    temp_cols['{year}_{master}'.format(year=year, master=master)]['journal'].append(count_info[1][counter])
                    temp_cols['{year}_{master}'.format(year=year, master=master)]['y-val'].append(num)
                    temp_cols['{year}_{master}'.format(year=year, master=master)]['citations'].append(num_cites)

for year_master, data in temp_cols.items():
    for header, info in data.items():
        column_name = '{year_master}_{header}_gapminder_grid'.format(year_master=year_master, header=header)
        a_column = Column(info, column_name)
        columns.append(a_column)

grid = Grid(columns)

In [114]:
# list of years
years = list(year_collections.keys())

# list of masters
masters = list(groups_by_master.keys())

temp_cols = {}
for year in years:
    for master in masters:
        temp_cols['{year}_{master}'.format(year=year, master=master)] = {'keyword': [], 'journal': [], 'y-val': []}
    
    data = year_collections[year]
    for keyword, num_citations in data.items():
        if keyword != 'traces':
            for num, count_info in num_citations.items():
                for counter, num_cites in enumerate(count_info[0]):
                    master = groups_by_keyword[keyword]
                    temp_cols['{year}_{master}'.format(year=year, master=master)]['keyword'].append(keyword+'<br>Number of Citations: '+num_cites)
                    temp_cols['{year}_{master}'.format(year=year, master=master)]['journal'].append(count_info[1][counter])
                    temp_cols['{year}_{master}'.format(year=year, master=master)]['y-val'].append(num)

In [115]:
temp_cols

{'2007_(text-belief consistency effect': {'keyword': [],
  'journal': [],
  'y-val': []},
 '2007_“Scribbling” dynamics': {'keyword': [], 'journal': [], 'y-val': []},
 '2007_3D shape perception': {'keyword': [], 'journal': [], 'y-val': []},
 '2007_a clear distinction between normative systems and competence theories': {'keyword': [],
  'journal': [],
  'y-val': []},
 '2007_ability-related beliefs': {'keyword': [], 'journal': [], 'y-val': []},
 '2007_abnormalities': {'keyword': [], 'journal': [], 'y-val': []},
 '2007_abstract theoretical challenges': {'keyword': [],
  'journal': [],
  'y-val': []},
 '2007_abstracted beyond sensory constraints': {'keyword': [],
  'journal': [],
  'y-val': []},
 '2007_abstraction': {'keyword': [], 'journal': [], 'y-val': []},
 '2007_abuse': {'keyword': [], 'journal': [], 'y-val': []},
 '2007_Academic Performance': {'keyword': [], 'journal': [], 'y-val': []},
 '2007_acceptance': {'keyword': [], 'journal': [], 'y-val': []},
 '2007_Accessibility': {'keyword':

In [116]:
figure = {
    'data': [],
    'layout': {},
    'frames': [],
}

# fill in most of layout
figure['layout']['xaxis'] = {'title': 'Journal', 'gridcolor': '#FFFFFF'}
figure['layout']['yaxis'] = {'title': 'Num of Citations', 'gridcolor': '#FFFFFF'}
figure['layout']['hovermode'] = 'closest'
figure['layout']['plot_bgcolor'] = 'rgb(223, 232, 243)'

In [117]:
sliders_dict = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 20},
        'prefix': 'Year:',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300, 'easing': 'cubic-in-out'},
    'pad': {'b': 10, 't': 50},
    'len': 0.9,
    'x': 0.1,
    'y': 0,
    'steps': []
}

In [118]:
figure['layout']['updatemenus'] = [
    {
        'buttons': [
            {
                'args': [None, {'frame': {'duration': 500, 'redraw': False},
                         'fromcurrent': True, 'transition': {'duration': 300, 'easing': 'quadratic-in-out'}}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate',
                'transition': {'duration': 0}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }
]

In [119]:
year = '2007'
for master in masters:
    year_master = '{year}_{master}'.format(year=year, master=master)
    data_dict = {
        'x': temp_cols[year_master]['journal'],
        'y': temp_cols[year_master]['y-val'],
        'mode': 'markers',
        'text': temp_cols[year_master]['keyword'],
        'marker': {
            'size': 8
        },
        'name': master
    }
    figure['data'].append(data_dict)

In [120]:
figure['data']

[{'x': [],
  'y': [],
  'mode': 'markers',
  'text': [],
  'marker': {'size': 8},
  'name': '(text-belief consistency effect'},
 {'x': [],
  'y': [],
  'mode': 'markers',
  'text': [],
  'marker': {'size': 8},
  'name': '“Scribbling” dynamics'},
 {'x': [],
  'y': [],
  'mode': 'markers',
  'text': [],
  'marker': {'size': 8},
  'name': '3D shape perception'},
 {'x': [],
  'y': [],
  'mode': 'markers',
  'text': [],
  'marker': {'size': 8},
  'name': 'a clear distinction between normative systems and competence theories'},
 {'x': [],
  'y': [],
  'mode': 'markers',
  'text': [],
  'marker': {'size': 8},
  'name': 'ability-related beliefs'},
 {'x': [],
  'y': [],
  'mode': 'markers',
  'text': [],
  'marker': {'size': 8},
  'name': 'abnormalities'},
 {'x': [],
  'y': [],
  'mode': 'markers',
  'text': [],
  'marker': {'size': 8},
  'name': 'abstract theoretical challenges'},
 {'x': [],
  'y': [],
  'mode': 'markers',
  'text': [],
  'marker': {'size': 8},
  'name': 'abstracted beyond sen

In [112]:
plot(figure)

'file:///Users/squinto/Downloads/cwsl_forecasting/temp-plot.html'

In [121]:
for year in years:
    frame = {'data': [], 'name': year}
    for master in masters:
        year_master = '{year}_{master}'.format(year=year, master=master)
        data_dict = {
            'x': temp_cols[year_master]['journal'],
            'y': temp_cols[year_master]['y-val'],
            'mode': 'markers',
            'text': temp_cols[year_master]['keyword'],
            'marker': {
                'size': 8
            },
            'name': master
        }
        frame['data'].append(data_dict)

    figure['frames'].append(frame)
    slider_step = {'args': [
        [year],
        {'frame': {'duration': 300, 'redraw': True},
         'mode': 'immediate',
       'transition': {'duration': 300}}
     ],
     'label': year,
     'method': 'animate'}
    sliders_dict['steps'].append(slider_step)

figure['layout']['sliders'] = [sliders_dict]

In [122]:
plot(figure)

'file:///Users/squinto/Downloads/cwsl_forecasting/temp-plot.html'

In [11]:
import sys
from itertools import groupby

master_nodes = {}
for year, keyword_dict in year_collections.items():
    L = sorted(keyword_dict.keys())
    result = []

    for key,group in groupby(L, lambda x: x.split(' ')[0]):
        result.append(list(group))
    
    print(result)

[['affect'], ['affective signalling', 'affective transformations'], ['attention'], ['categories of language'], ['categorization'], ['culture'], ['episteme'], ['epistemic cognition'], ['epistemology'], ['evolution'], ['evolutionary psychology'], ['functionalism'], ['functional~emotional hypothesis'], ['grammar'], ['intention'], ['joint attention'], ['language development', 'language learning'], ['linguistics'], ['mindlessness'], ['mind–body problem'], ['mind–brain mechanisms'], ['pattern', 'pattern formation'], ['psycholinguistics'], ['psychological theory'], ['self-conscious emotion'], ['self-knowledge'], ['self-organization'], ['self-organizing complex systems'], ['self-regulation'], ['self-related'], ['selfhood'], ['shared concrete similarity'], ['similarity'], ['social cognition', 'social comparison'], ['wittgenstein']]
[['abnormality'], ['action-based framework'], ['action–inaction distinction'], ['adaptive help seeking', 'adaptive learning', 'adaptive teaching'], ['addiction'], ['

In [12]:
import sys
from itertools import groupby

master_nodes = {}
for year, keyword_dict in year_collections.items():
    L = sorted(keyword_dict.keys())
    result = []

    for key,group in groupby(L, lambda x: x.split(' ')[0]):
        result.append(list(group))
    
    for l in result:
        mn = l[0]
        if mn not in master_nodes:
            master_nodes[mn] = {}
            mn_dict = master_nodes[mn]
            mn_dict['x'] = []
            mn_dict['y'] = []
            mn_dict['text'] = []
        else:
            mn_dict = master_nodes[mn]
            
        for keyword in l:
            for num_citations in keyword_dict[keyword].keys():
                mn_dict['x'].append(year)
                mn_dict['y'].append(num_citations)
                mn_dict['text'].append(keyword)
                
sorted_mn = sorted(master_nodes.items())